In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
import numpy as np
import cv2
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mtcnn import MTCNN
import mediapipe as mp

In [2]:
# Initialize MediaPipe face mesh (for training and detection)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


In [3]:
# Initialize MTCNN (only for registration)
mtcnn_detector = MTCNN()

In [4]:
def register_user_with_mtcnn(user_id, dataset_dir, num_images=5):
    """Register new user using MTCNN for face detection"""
    user_folder = os.path.join(dataset_dir, user_id)
    os.makedirs(user_folder, exist_ok=True)
    
    cap = cv2.VideoCapture(0)
    print(f"Registering user {user_id}. Please position your face in the frame.")
    
    captured_images = 0
    while captured_images < num_images and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Detect faces with MTCNN
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mtcnn_detector.detect_faces(frame_rgb)
        
        if len(results) > 0:
            # Get the largest face
            x, y, w, h = results[0]['box']
            face = frame_rgb[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (160, 160))
            
            # Save image
            img_path = os.path.join(user_folder, f"{user_id}_{captured_images}.jpg")
            cv2.imwrite(img_path, cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR))
            captured_images += 1
            print(f"Captured image {captured_images}/{num_images}")
            
            # Show countdown
            for i in range(3, 0, -1):
                ret, frame = cap.read()
                if ret:
                    cv2.putText(frame, f"Next in {i}...", (10, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow('Registration', frame)
                    cv2.waitKey(1000)
        
        # Display instructions
        cv2.putText(frame, f"Captured: {captured_images}/{num_images}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow('Registration', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Registration complete for {user_id}")

In [5]:
def extract_mediapipe_landmarks(image):
    """Extract face landmarks using MediaPipe"""
    results = face_mesh.process(image)
    if results.multi_face_landmarks:
        landmarks = []
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])
        return np.array(landmarks)
    return None

In [6]:
def load_dataset_with_mediapipe(dataset_dir):
    """Load dataset using MediaPipe for landmark extraction"""
    X, y = [], []
    print(f"Loading dataset from {dataset_dir}")
    
    for label in os.listdir(dataset_dir):
        label_path = os.path.join(dataset_dir, label)
        if not os.path.isdir(label_path):
            continue
            
        print(f"Processing {label}...")
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
                
            # Read and process image
            image = cv2.imread(img_path)
            if image is None:
                continue
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            landmarks = extract_mediapipe_landmarks(image_rgb)
            
            if landmarks is not None:
                X.append(landmarks)
                y.append(label)
    
    print(f"Loaded {len(X)} samples")
    return np.array(X), np.array(y)

In [7]:
def build_model(input_shape, num_classes):
    """Build a simple neural network model"""
    model = Sequential([
        Dense(256, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()
    
    return model

In [8]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization

def build_model_v1(input_shape, num_classes):
    """Model dengan BatchNorm dan L2 Regularization"""
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [9]:
from tensorflow.keras.layers import LeakyReLU

def build_model_v2(input_shape, num_classes):
    """Model dengan LeakyReLU activation"""
    model = Sequential([
        Dense(128),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),
        Dense(64),
        LeakyReLU(alpha=0.1),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [10]:
from tensorflow.keras.initializers import HeNormal

def build_model_v3(input_shape, num_classes):
    """Model dengan He Weight Initialization"""
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape, 
              kernel_initializer=HeNormal()),
        Dropout(0.3),
        Dense(64, activation='relu', 
              kernel_initializer=HeNormal()),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [11]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch):
    """Learning rate scheduler"""
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lr = initial_lr * (drop ** np.floor((1+epoch)/epochs_drop))
    return lr

def build_model_v4(input_shape, num_classes):
    """Model dengan Learning Rate Scheduling"""
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model, LearningRateScheduler(lr_schedule)

In [12]:
from tensorflow.keras.layers import InputLayer

def build_model_v5(input_shape, num_classes):
    """Model dengan Input Layer eksplisit"""
    model = Sequential([
        InputLayer(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [13]:
def build_cnn_model(input_shape, num_classes):
    """Build CNN model with reshaped input"""
    model = Sequential([
        Reshape((input_shape[0]//3, 3, 1)),  # Reshape landmark data to 3D
        Conv1D(32, 3, activation='relu'),
        MaxPooling1D(2),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [14]:
def build_landmark_model(input_shape, num_classes):
    """Build an optimized model for MediaPipe landmark data"""
    model = Sequential([
        Dense(256, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [15]:
def extract_face_and_landmarks(image_rgb):
    """Extract face and landmarks using MediaPipe"""
    results = face_mesh.process(image_rgb)
    
    if results.multi_face_landmarks:
        # Get face bounding box from landmarks
        landmarks = results.multi_face_landmarks[0].landmark
        xs = [lm.x for lm in landmarks]
        ys = [lm.y for lm in landmarks]
        
        # Calculate bounding box
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)
        
        # Convert to pixel coordinates
        h, w, _ = image_rgb.shape
        x1, y1 = int(x_min * w), int(y_min * h)
        x2, y2 = int(x_max * w), int(y_max * h)
        
        # Extract landmarks
        landmark_features = []
        for landmark in landmarks:
            landmark_features.extend([landmark.x, landmark.y, landmark.z])
        
        return np.array(landmark_features), (x1, y1, x2-x1, y2-y1)
    
    return None, None

In [16]:
def realtime_detection_with_mediapipe(model, label_encoder):
    """Real-time face recognition using MediaPipe"""
    cap = cv2.VideoCapture(0)
    prev_time = time.time()
    
    # Create label mapping
    label_map = {i: name for i, name in enumerate(label_encoder.classes_)}
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Start timing for FPS calculation
        start_time = time.time()
        
        # Convert to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Get face landmarks
        landmarks, bbox = extract_face_and_landmarks(frame_rgb)
        
        if landmarks is not None:
            # Predict
            features = landmarks.reshape(1, -1)
            predictions = model.predict(features, verbose=0)
            idx = np.argmax(predictions)
            confidence = np.max(predictions) * 100
            
            # Get label
            label = label_map.get(idx, "Unknown")
            
            # Draw results
            x, y, w, h = bbox
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.1f}%)", (x, y - 10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Calculate and display FPS
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time
        
        fps_color = (0, 255, 0) if fps > 15 else (0, 255, 255) if fps > 10 else (0, 0, 255)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, fps_color, 2)
        
        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Main execution
if __name__ == "__main__":
    dataset_dir = './dataset_model_skripsi/'

    
    # User registration (using MTCNN)
    register_new = input("Register new user? (y/n): ").lower()
    if register_new == 'y':
        user_id = input("Enter user ID/NIM: ")
        register_user_with_mtcnn(user_id, dataset_dir)
    
    # Load dataset (using MediaPipe)
    X, y = load_dataset_with_mediapipe(dataset_dir)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42)
    
    # Hitung class weights
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
    class_weights = dict(enumerate(class_weights))

    # Build and train model
    model = build_model((X.shape[1],), len(label_encoder.classes_))
    model.fit(X_train, y_train, epochs=20, class_weight=class_weights, validation_data=(X_test, y_test))
    
    # Save model
    model.save('face_recognition_mediapipe.h5')
    print("Model trained and saved.")
    
    # Evaluate
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {acc*100:.2f}%")
    
    # Real-time detection (using MediaPipe)
    realtime_detection_with_mediapipe(model, label_encoder)

Loading dataset from ./dataset_model_skripsi/
Processing 2111231231...
Loaded 5 samples


c:\Users\jefta\Documents\Tugas Akhir Next js\prototype-leads\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       367,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 410,113 (1.56 MB)

...

Epoch 1/20


c:\Users\jefta\Documents\Tugas Akhir Next js\prototype-leads\.venv\Lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\jefta\Documents\Tugas Akhir Next js\prototype-leads\.venv\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 6.0655 - val_accuracy: 1.0000 - val_loss: 5.8107
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 1.0000 - loss: 5.8107 - val_accuracy: 1.0000 - val_loss: 5.5636
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 5.5636 - val_accuracy: 1.0000 - val_loss: 5.3244
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 5.3244 - val_accuracy: 1.0000 - val_loss: 5.0930
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 5.0930 - val_accuracy: 1.0000 - val_loss: 4.8696
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 4.8696 - val_accuracy: 1.0000 - val_loss: 4.6539
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 4.6539 - val_accuracy: 1.0000 - val_loss: 4.4460
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 4.4460 - val_accuracy: 1.0000 - val_loss: 4.2458
Epoch 9/20
1/

Model trained and saved.
Test accuracy: 100.00%


c:\Users\jefta\Documents\Tugas Akhir Next js\prototype-leads\.venv\Lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [ ]:
import pandas as pd
print(pd.Series(y).value_counts())